In [1]:
!pip install datasets --quiet
!pip install transformers --quiet
!pip install torch --quiet
!pip install emoji --quiet
!pip install sentencepiece --quiet
!pip install git+https://github.com/huggingface/accelerate --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 92.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 114.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 353.7/353.7 kB 16.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) .

In [2]:
import numpy as np 
import pandas as pd 
import warnings
import os
import re
import nltk
import emoji

from nltk.corpus import stopwords
from tqdm import tqdm

warnings.filterwarnings("ignore")

In [3]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
data = pd.read_csv('/content/drive/Shareddrives/DL + NLP/Proyecto DL + NLP/Entrega_Final/data/raw/combined_balanced.csv')

In [6]:
data

,subreddit,post,label
0,control,What do you do when most of a class passes and...,0
1,control,"Picking a chapter book for ""cool down time"" I'...",0
2,control,How can I prevent alumni from visiting me? As ...,0
3,control,Why I feel like a dumb student In elementary S...,0
4,control,Anonymous questionnaire on Social Anxiety Hell...,0
...,...,...,...
143995,bipolarreddit,Business travel w/ bipolar? Hi Friends -\n\nI'...,3
143996,bipolarreddit,Finding a job when depressed is impossible. Fi...,3
143997,bipolarreddit,Mania?? I was diagnosed with bipolar over 4 ye...,3
143998,bipolarreddit,Are any of you successfully managing your cond...,3


In [7]:
pattern_1 = "adhd"
pattern_2 = "anxiety"

filter_1 = data['subreddit'].str.contains(pattern_1)
filter_2 = data['subreddit'].str.contains(pattern_2)

data_1 = data[filter_1]
data_2 = data[filter_2]

In [8]:
frames = [data_1, data_2]

data = pd.concat(frames)

In [9]:
data

,subreddit,post,label
48000,adhd,Has anyone tried the GeneSight genetic test fo...,1
48001,adhd,I was doing so well... I was diagnosed back in...,1
48002,adhd,Is it possible to have ADHD and not typically ...,1
48003,adhd,"Is it possible to treat ADD, anxiety, and depr...",1
48004,adhd,Everything seems very complicated. I was diagn...,1
...,...,...,...
47995,anxiety,Anyone try TMS? I am starting TMS on Monday fr...,2
47996,anxiety,So I’ve been thinking of getting Propranolol f...,2
47997,anxiety,"Organic, natural, vegan CBD Hey guys, you've p...",2
47998,anxiety,Anxiety or MS? 25/M. Over the last four month...,2


In [10]:
train_text_df  = data.rename(columns={'subreddit': 'entities', 'post': 'text', 'label': 'labels'})

In [11]:
train_text_df ["id"] = train_text_df .index + 1

In [12]:
train_text_df ['entities'] = train_text_df ['entities'].str.strip('()').str.split(',')

In [13]:
from sklearn.utils import shuffle
train_text_df = shuffle(train_text_df)

In [14]:
train_text_df.drop(['labels'], axis=1, inplace=True)

In [15]:
train_text_df = train_text_df.reset_index(drop=True)

In [16]:
nltk.download('stopwords')
stop = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [17]:
train_text_df['text'] = train_text_df['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
train_text_df['text'] = train_text_df['text'].str.lower()
train_text_df['text'] = train_text_df['text'].apply(lambda s: emoji.replace_emoji(s, ''))
train_text_df['text'] = train_text_df['text'].apply(lambda x: re.split('https:\/\/.*', str(x))[0])
train_text_df['text'] = train_text_df['text'].str.replace('[^a-zA-Z\ ]', '')

In [18]:
train_text_df 

,entities,text,id
0,[anxiety],i want bored this short thing say i want advic...,47718
1,[adhd],college coping tactics the new semester starte...,62222
2,[adhd],got first batch medication add i absolutely te...,56232
3,[anxiety],panic attacks hi all anxiety pretty rough late...,42697
4,[adhd],i know mom i got diagnosed add she knows i add...,57611
...,...,...,...
31995,[anxiety],growing anxiety good enough for context im you...,45992
31996,[adhd],newly diagnosed my son diagnosed adhd he nearl...,55362
31997,[adhd],monday again another chance right time another...,52204
31998,[anxiety],why i goad myself woke morning use bathroom i ...,38382


In [19]:
import ast

train_text_df['entities'] = train_text_df['entities'].apply(lambda x:ast.literal_eval(str(x)))

print(train_text_df['entities'].values[0])

['anxiety']


In [20]:
train_text_df = train_text_df[:]

In [21]:
import datasets
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding
from transformers import BigBirdForTokenClassification, BigBirdTokenizerFast
from torch import cuda
import torch

In [22]:
config = {'model_name': 'google/bigbird-roberta-base',
         'max_length': 150,
         'train_batch_size':4,
         'valid_batch_size':4,
         'epochs':5,
         'learning_rate':5e-05,
         'max_grad_norm':10,
          'warmup':0.1,
          "grad_acc":8,
          "model_save_path":"big-bird",
         'device': 'cuda' if cuda.is_available() else 'cpu'}

output_labels = ['anxiety', 'adhd']

labels_to_ids = {v:k for k,v in enumerate(output_labels)}
ids_to_labels = {k:v for k,v in enumerate(output_labels)}

In [23]:
train_text_df['labels'] = train_text_df['entities'].apply(lambda x: [labels_to_ids[i] for i in x])

In [24]:
train_text_df.head()

,entities,text,id,labels
0,[anxiety],i want bored this short thing say i want advic...,47718,[0]
1,[adhd],college coping tactics the new semester starte...,62222,[1]
2,[adhd],got first batch medication add i absolutely te...,56232,[1]
3,[anxiety],panic attacks hi all anxiety pretty rough late...,42697,[0]
4,[adhd],i know mom i got diagnosed add she knows i add...,57611,[1]


In [25]:
tokenizer = BigBirdTokenizerFast.from_pretrained(config['model_name'])
model = BigBirdForTokenClassification.from_pretrained(config['model_name'],
                                                     num_labels=len(output_labels))

Some weights of the model checkpoint at google/bigbird-roberta-base were not used when initializing BigBirdForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BigBirdForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BigBirdForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BigBirdForTokenClassification were no

In [26]:
converted = tokenizer(train_text_df.loc[0].values[1].split(),
                      is_split_into_words=True, return_offsets_mapping=True)

In [27]:
ix = 0
for i,j in zip(tokenizer.convert_ids_to_tokens(converted['input_ids']), converted['offset_mapping']):
    print(i, j)
    ix += 1
    if ix == 15:
        break

[CLS] (0, 0)
▁i (0, 1)
▁want (0, 4)
▁bored (0, 5)
▁this (0, 4)
▁short (0, 5)
▁thing (0, 5)
▁say (0, 3)
▁i (0, 1)
▁want (0, 4)
▁advice (0, 6)
▁lately (0, 6)
▁life (0, 4)
▁non (0, 3)
stop (3, 7)


In [28]:
def tokenizer_data(example):
    encoding = tokenizer(example['text'].split(),
                         is_split_into_words=True,
                         truncation=True,
                         padding='max_length', 
                         return_offsets_mapping=True,
                         max_length=config['max_length'])
    i = 0
    labels = example['labels']
    encoded_labels = np.ones(len(encoding["offset_mapping"]), dtype=int) * -100
    for idx, mapping in enumerate(encoding["offset_mapping"]):
        if mapping[0] == 0 and mapping[1] != 0:
            try:
                encoded_labels[idx] = labels[i]
            except:
                pass
            i += 1
    item = {key: torch.as_tensor(val) for key, val in encoding.items()}
    item['labels'] = torch.as_tensor(encoded_labels)
    return item

In [29]:
dataset = datasets.Dataset.from_pandas(train_text_df)

In [30]:
dataset = dataset.train_test_split(test_size=0.1)
dataset

DatasetDict({
    train: Dataset({
        features: ['entities', 'text', 'id', 'labels'],
        num_rows: 28800
    })
    test: Dataset({
        features: ['entities', 'text', 'id', 'labels'],
        num_rows: 3200
    })
})

In [31]:
text = dataset['train'][1]

In [32]:
converted = tokenizer_data(text)

converted

{'input_ids': tensor([   65,  1413,  8491, 41009, 44894, 19097,   321,   526,  1861, 13080,
          1413,  4341,  2788,  2174,  1712,  1618,  1413,  2641,  3005,   457,
         11150,  6069,   604,    66,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,   

In [33]:
converted

i=0
for token, label in zip(tokenizer.convert_ids_to_tokens(converted["input_ids"]), 
                        converted["labels"]):
    print(token, label, converted['offset_mapping'][i])
    i+=1
    if i == 15:
        break

[CLS] tensor(-100) tensor([0, 0])
▁i tensor(0) tensor([0, 1])
▁fake tensor(-100) tensor([0, 4])
▁yawn tensor(-100) tensor([0, 4])
▁unintentionally tensor(-100) tensor([ 0, 15])
▁realise tensor(-100) tensor([0, 7])
▁ tensor(-100) tensor([0, 1])
ive tensor(-100) tensor([0, 3])
▁done tensor(-100) tensor([0, 4])
▁afterwards tensor(-100) tensor([ 0, 10])
▁i tensor(-100) tensor([0, 1])
▁wonder tensor(-100) tensor([0, 6])
▁anyone tensor(-100) tensor([0, 6])
▁else tensor(-100) tensor([0, 4])
▁kind tensor(-100) tensor([0, 4])


In [34]:
dataset = dataset.map(tokenizer_data)

dataset

Map:   0%|          | 0/28800 [00:00<?, ? examples/s]

Map:   0%|          | 0/3200 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['entities', 'text', 'id', 'labels', 'input_ids', 'attention_mask', 'offset_mapping'],
        num_rows: 28800
    })
    test: Dataset({
        features: ['entities', 'text', 'id', 'labels', 'input_ids', 'attention_mask', 'offset_mapping'],
        num_rows: 3200
    })
})

In [35]:
dataset.set_format(type='torch', columns=['input_ids', 'attention_mask',
                                         'labels'])

dataset

DatasetDict({
    train: Dataset({
        features: ['entities', 'text', 'id', 'labels', 'input_ids', 'attention_mask', 'offset_mapping'],
        num_rows: 28800
    })
    test: Dataset({
        features: ['entities', 'text', 'id', 'labels', 'input_ids', 'attention_mask', 'offset_mapping'],
        num_rows: 3200
    })
})

In [42]:
for a in dataset['train']:
    if a['input_ids'].shape[0] != a['attention_mask'].shape[0]:
        print(a)
        break

In [46]:
trainer_args = TrainingArguments(report_to='none',
                                output_dir="/content/drive/Shareddrives/DL + NLP/Proyecto DL + NLP/Entrega_Final/output/classificator/BigBird/Anxiety+ADHD_V2",
                                num_train_epochs=config['epochs'],
                                evaluation_strategy ='epoch',
                                per_device_train_batch_size=config['train_batch_size'],
                                per_device_eval_batch_size=config['valid_batch_size'],
                                fp16=True,
                                save_strategy = "epoch",
                                warmup_ratio= config['warmup'],
                                gradient_accumulation_steps=config['grad_acc'],
                                logging_strategy="epoch",
                                save_total_limit=1
                                )

trainer = Trainer(model=model,
                  args=trainer_args, 
                  train_dataset = dataset['train'],
                  eval_dataset=dataset['test'],
                  tokenizer=tokenizer)

In [47]:
trainer.train()

You're using a BigBirdTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Attention type 'block_sparse' is not possible if sequence_length: 150 <= num global tokens: 2 * config.block_size + min. num sliding tokens: 3 * config.block_size + config.num_random_blocks * config.block_size + additional buffer: config.num_random_blocks * config.block_size = 704 with config.block_size = 64, config.num_random_blocks = 3. Changing attention type to 'original_full'...


Epoch,Training Loss,Validation Loss
1,0.247400,0.154133
2,0.120600,0.155327
3,0.078100,0.146349
4,0.044000,0.179440
5,0.023900,0.191575


TrainOutput(global_step=4500, training_loss=0.10281156158447266, metrics={'train_runtime': 2843.0953, 'train_samples_per_second': 50.649, 'train_steps_per_second': 1.583, 'total_flos': 1.1099997648e+16, 'train_loss': 0.10281156158447266, 'epoch': 5.0})

In [48]:
device = config['device']

In [49]:
trainer.model.eval()
def inference(sentence):
    inputs = tokenizer(sentence.split(),
                        is_split_into_words=True, 
                        return_offsets_mapping=True, 
                        padding='max_length', 
                        truncation=True, 
                        max_length=200,
                        return_tensors="pt")

    # move to gpu
    ids = inputs["input_ids"].to(device)
    mask = inputs["attention_mask"].to(device)
    # forward pass
    outputs = trainer.model(input_ids=ids, attention_mask=mask, return_dict=False)
#     print(outputs)
    logits = outputs[0]
    
    active_logits = logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
    flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size*seq_len,) - predictions at the token level
    print(logits.shape, active_logits.shape, flattened_predictions.shape)
    tokens = tokenizer.convert_ids_to_tokens(ids.squeeze().tolist())
    token_predictions = [ids_to_labels[i] for i in flattened_predictions.cpu().numpy()]
    wp_preds = list(zip(tokens, token_predictions)) # list of tuples. Each tuple = (wordpiece, prediction)

    prediction = []
    out_str = []
    off_list = inputs["offset_mapping"].squeeze().tolist()
    for idx, mapping in enumerate(off_list):
#         print(mapping, token_pred[1], token_pred[0],"####")

#         only predictions on first word pieces are important
        if mapping[0] == 0 and mapping[1] != 0:
#             print(mapping, token_pred[1], token_pred[0])
            prediction.append(wp_preds[idx][1])
            out_str.append(wp_preds[idx][0])
        else:
            if idx == 1:
                prediction.append(wp_preds[idx][1])
                out_str.append(wp_preds[idx][0])
            continue
    return prediction, out_str

In [50]:
text_1= 'Ive never taken it to a doctor, but Ive heard that it could help for anxiety, but I dont want to take it to the point where I am constantly afraid of dying.'
pred_1, _ = inference(text_1)
print(pred_1[0])

torch.Size([1, 200, 2]) torch.Size([200, 2]) torch.Size([200])
anxiety


In [51]:
text_2= 'I am not sure if this is a sign that I am not sick or if I am just lazy and have no sense of direction.'
pred_2, _ = inference(text_2)
print(pred_2[0])

torch.Size([1, 200, 2]) torch.Size([200, 2]) torch.Size([200])
adhd
